In [105]:
import os
import json
import boto3
import pandas as pd
from io import StringIO

In [106]:
s3 = boto3.client('s3')

Load both the csv file from **Part 1** `pr.data.0.Current` and the json file from **Part 2**

In [107]:
obj = s3.get_object( 
    Bucket="questdata",
    Key="pr.data.0.Current"
)
StringData = StringIO(obj['Body'].read().decode('utf-8'))
df_part_1 = pd.read_csv(StringData, sep="\t")
df_part_1.rename(
    columns = {
        old_name:old_name.strip().title()
        for old_name in df_part_1.columns.tolist()
    }, 
    inplace = True
)

In [108]:
obj = s3.get_object( 
    Bucket="questdata",
    Key="api.json"
)
j = json.loads(obj['Body'].read())
df_part_2 = pd.DataFrame.from_records(j['data'])
df_part_2['Year'] = pd.to_numeric(df_part_2['Year'])

Using the dataframe from the population data API **Part 2**, generate the mean and the standard deviation of the US population across the years [2013, 2018] inclusive.

**Answer**

In [109]:
df_part_2

,ID Nation,Nation,ID Year,Year,Population,Slug Nation
0,01000US,United States,2020,2020,326569308,united-states
1,01000US,United States,2019,2019,324697795,united-states
2,01000US,United States,2018,2018,322903030,united-states
3,01000US,United States,2017,2017,321004407,united-states
4,01000US,United States,2016,2016,318558162,united-states
5,01000US,United States,2015,2015,316515021,united-states
6,01000US,United States,2014,2014,314107084,united-states
7,01000US,United States,2013,2013,311536594,united-states


The folks at datausa are currently having issues with their API:
![Broken API](BrokenAPI.png)

I was able to get a payload from the base slug in the README. It's the dataframe above this cell. I won't be able to "generate the mean and the standard deviation" since I have no idea how to get more granular with this API. 🙄

Using the dataframe from the time-series (Part 1), For every series_id, find the *best year*: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year.

**Answer**

After grouping by the `"Series_Id"` and `"Year"`, we sum the values. This gives us the "total" value per `"Series_Id"` and `"Year"`. The pandas series is then reduced to a dataframe. The new dataframe is then indexed by the largest value per `"Year"`.

In [142]:
df = df_part_1.groupby(
    ["Series_Id", "Year"]
)["Value"].sum().reset_index(
    level = [0 , 1]
)
df.loc[df.groupby('Series_Id')['Value'].idxmax()]

,Series_Id,Year,Value
17,PRS30006011,2012,9.500
54,PRS30006012,2021,13.100
59,PRS30006013,1998,733.290
99,PRS30006021,2010,14.200
127,PRS30006022,2010,11.100
...,...,...,...
7673,PRS88003192,2002,285.200
7720,PRS88003193,2021,620.416
7734,PRS88003201,2007,21.700
7776,PRS88003202,2021,26.500


Using both dataframes from Part 1 and Part 2, generate a report that will provide the `value` for `series_id = PRS30006032` and `period = Q01` and the `population` for that given year (if available in the population dataset)

**Answer**

Based on how the question is written, I'm assuming that you want join the 2 dataframes together. Here, I've joined the dataframes through a `'right'` join. This means that each year in part 1 is joined with each year in part 2. If the year isn't in part 2, it's not in this dataframe.

In [104]:
df_part_1.merge(df_part_2, on='Year', how='right')

,Series_Id,Year,Period,Value,Footnote_Codes,ID Nation,Nation,ID Year,Population,Slug Nation
0,PRS30006011,2020,Q01,-0.800,NaN,01000US,United States,2020,326569308,united-states
1,PRS30006011,2020,Q02,-8.300,NaN,01000US,United States,2020,326569308,united-states
2,PRS30006011,2020,Q03,-6.400,NaN,01000US,United States,2020,326569308,united-states
3,PRS30006011,2020,Q04,-4.700,NaN,01000US,United States,2020,326569308,united-states
4,PRS30006011,2020,Q05,-5.100,NaN,01000US,United States,2020,326569308,united-states
...,...,...,...,...,...,...,...,...,...,...
9835,PRS88003203,2013,Q01,100.692,NaN,01000US,United States,2013,311536594,united-states
9836,PRS88003203,2013,Q02,100.472,NaN,01000US,United States,2013,311536594,united-states
9837,PRS88003203,2013,Q03,101.126,NaN,01000US,United States,2013,311536594,united-states
9838,PRS88003203,2013,Q04,101.874,NaN,01000US,United States,2013,311536594,united-states
